Import Packages

In [1]:
import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiLineString, LineString
from shapely.ops import nearest_points
import math
import numpy as np

Load Data from our API Calls

In [2]:
trunkroute = pd.read_csv("Bus_RoutesStopsServices/trunkroute.csv")
transport_node_bus_202407 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202407.csv")
transport_node_bus_202408 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202408.csv")
transport_node_bus_202409 = pd.read_csv("Passenger_Volume_By_Bus_Stop/transport_node_bus_202409.csv")

print(trunkroute.head())

  ServiceNo Operator  Direction  StopSequence  BusStopCode  Distance  \
0        10     SBST          1             1        75009       0.0   
1        10     SBST          1             2        76059       0.6   
2        10     SBST          1             3        76069       1.1   
3        10     SBST          1             4        96289       2.3   
4        10     SBST          1             5        96109       2.7   

  WD_FirstBus WD_LastBus SAT_FirstBus SAT_LastBus SUN_FirstBus SUN_LastBus  \
0        0500       2300         0500        2300         0500        2300   
1        0502       2302         0502        2302         0502        2302   
2        0504       2304         0504        2304         0503        2304   
3        0508       2308         0508        2309         0507        2308   
4        0509       2310         0509        2311         0508        2309   

          RoadName           Description  Latitude   Longitude  
0  Tampines Ctrl 1          Tampi

MRT line mapping
- mrt_lines mapping
- function to find MRT line based on bus stop description

In [3]:
#mapping of stations to MRT lines
mrt_lines = {
    'Thomson-East Coast Line': ['WOODLANDS NORTH MRT STATION','WOODLANDS MRT STATION','WOODLANDS SOUTH MRT STATION', 'SPRINGLEAF MRT STATION','LENTOR MRT STATION','MAYFLOWER MRT STATION','BRIGHT HILL MRT STATION',
                                'UPPER THOMSON MRT STATION','CALDECOTT MRT STATION','STEVENS MRT STATION','NAPIER MRT STATION','ORCHARD BOULEVARD MRT STATION','ORCHARD MRT STATION','GREAT WORLD MRT STATION',
                                'HAVELOCK MRT STATION','OUTRAM PARK MRT STATION','MAXWELL MRT STATION','SHENTON WAY MRT STATION','MARINA BAY MRT STATION','GARDENS BY THE BAY MRT STATION','TANJONG RHU MRT STATION',
                                'KATONG PARK MRT STATION','TANJONG KATONG MRT STATION','MARINE PARADE MRT STATION','MARINE TERRACE MRT STATION','SIGLAP MRT STATION','BAYSHORE MRT STATION'],
    'North-South Line': ['JURONG EAST MRT STATION','BUKIT BATOK MRT STATION','BUKIT GOMBAK MRT STATION','CHOA CHU KANG MRT STATION','YEW TEE MRT STATION','KRANJI MRT STATION','MARSILING MRT STATION','WOODLANDS MRT STATION',
                         'ADMIRALTY MRT STATION','SEMBAWANG MRT STATION','CANBERRRA MRT STATION','YISHUN MRT STATION','KHATIB MRT STATION','YIO CHU KANG MRT STATION','ANG MO KIO MRT STATION','BISHAN MRT STATION',
                         'BRADDELL MRT STATION','TOA PAYOH MRT STATION','NOVENA MRT STATION','NEWTON MRT STATION','ORCHARD MRT STATION','SOMERSET MRT STATION','DHOBY GHAUT MRT STATION','CITY HALL MRT STATION',
                         'RAFFLES PLACE MRT STATION','MARINA BAY MRT STATION','MARINA SOUTH PIER MRT STATION'],
    'Circle Line': ['DHOBY GHAUT MRT STATION', 'BRAS BASAH MRT STATION', 'ESPLANADE MRT STATION', 'PROMENADE MRT STATION','NICOLL HIGHWAY MRT STATION','STADIUM MRT STATION',
                    'MOUNTBATTEN MRT STATION','DAKOTA MRT STATION','PAYA LEBAR MRT STATION','MACPHERSON MRT STATION','TAI SENG MRT STATION','BARTLEY MRT STATION','SERANGOON MRT STATION','LORONG CHUAN MRT STATION',
                    'BISHAN MRT STATION','MARYMOUNT MRT STATION','CALDECOTT MRT STATION','BOTANIC GARDENS MRT STATION','FARRER ROAD MRT STATION','HOLLAND VILLAGE MRT STATION','BUONA VISTA MRT STATION','ONE-NORTH MRT STATION',
                    'KENT RIDGE MRT STATION', 'HAW PAW VILLA MRT STATION', 'PASIR PANJANG MRT STATION', 'LABRADOR PARK MRT STATION', 'TELOK BLANGAH MRT STATION','HABOURFRONT MRT STATION'],
    'Circle Line.': ['PROMENADE MRT STATION','BAYFRONT MRT STATION', 'MARINA BAY MRT STATION'],
    'North-East Line': ['HARBOURFRONT MRT STATION','OUTRAM PARK MRT STATION','CHINATOWN MRT STATION','CLARKE QUAY MRT STATION','DHOBY GHAUT MRT STATION','LITTLE INDIA MRT STATION','FARRER PARK MRT STATION','BOON KENG MRT STATION',
                        'POTONG PASIR MRT STATION','WOODLEIGH MRT STATION','SERANGOON MRT STATION','KOVAN MRT STATION','HOUGANG MRT STATION','BUANGKOK MRT STATION','SENGKANG MRT STATION','PUNGGOL MRT STATION'],
    'Downtown Line': ['BUKIT PANJANG MRT STATION', 'CASHEW MRT STATION', 'HILLVIEW MRT STATION', 'HUME MRT STATION', 'BEAUTY WORLD MRT STATION', 'KING ALBERT PARK MRT STATION', 'SIXTH AVENUE','TAN KAH KEE MRT STATION',
                      'BOTANIC GARDENS MRT STATION', 'STEVENS MRT STATION', 'NEWTON MRT STATION', 'LITTLE INDIA MRT STATION', 'ROCHOR MRT STATION', 'BUGIS MRT STATION', 'PROMENADE MRT STATION', 'BAYFRONT MRT STATION',
                      'DOWNTOWN MRT STATION', 'TELOK AYER MRT STATION', 'CHINATOWN MRT STATION', 'FORT CANNING MRT STATION', ' BENCOOLEN MRT STATION', 'JALAN BESAR MRT STATION', 'BENDEMEER MRT STATION', 'GEYLANG  BAHRU MRT STATION',
                      'MATTAR MRT STATION', 'MACPHERSON MRT STATION', 'UBI MRT STATION', 'KAKI BUKIT MRT STATION', ' BEDOK NORTH MRT STATION', 'BEDOK RESERVOIR MRT STATION', 'TAMPINES WEST MRT STATION',
                      'TAMPINES MRT STATION', 'TAMPINES EAST MRT STATION', 'UPPER CHANGI MRT STATION', 'EXPO MRT STATION'],
    'East-West Line': ['PASIR RIS MRT STATION', 'TAMPINES MRT STATION', 'SIMEI MRT STATION', 'TANAH MERAH MRT STATION','BEDOK MRT STATION', 'KEMBANGAN MRT STATION', 'EUNOS MRT STATION', 'PAYA LEBAR MRT STATION',
                       'ALJUNIED MRT STATION', 'KALLANG MRT STATION', ' LAVENDER MRT STATION','BUGIS MRT STATION', 'CITY HALL MRT STATION', 'RAFFLES PLACE MRT STATION','TANJONG PAGAR MRT STATION', 'OUTRAM PARK MRT STATION',
                       'TIONG BAHRU MRT STATION', 'REDHILL MRT STATION', 'QUEENSTOWN MRT STATION', 'COMMONWEALTH MRT STATION','BUONA VISTA MRT STATION', 'DOVER MRT STATION', 'CLEMENTI MRT STATION', 'JURONG EAST MRT STATION',
                       'CHINESE GARDEN MRT STATION', 'LAKESIDE MRT STATION', 'BOON LAY MRT STATION', 'PIONEER MRT STATION','JOO KON MRT STATION',' GUL CIRCLE MRT STATION', 'TUAS CRESCENT MRT STATION', 
                       'TUAS WEST ROAD MRT STATION', 'TUAS LINK MRT STATION'],
    'East-West Line.':['TANAH MERAH MRT STATION','EXPO MRT STATION', 'CHANGI AIRPORT MRT STATION']
}

def find_mrt_line(description, mrt_lines):

    matching_lines = []

    for line, stations in mrt_lines.items():
        for station in stations:
            station_name = station.replace('MRT STATION', '').strip().lower() 
            if station_name in description.lower(): 
                matching_lines.append(line)
    return matching_lines if matching_lines else None

Weekday Bus Stop Passenger Volume Data

relevant columns:
- PT_CODE: bus stop code
- average_passenger_volume: the sum of tap in & tap out in the 3 months, divided by 3

In [4]:
#for each month, day type, and bus stop - get sum of tap in + tap out
jul_busstop_vol = transport_node_bus_202407.groupby(['PT_TYPE', 'PT_CODE', 'DAY_TYPE']).agg(
    july_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    july_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

aug_busstop_vol = transport_node_bus_202408.groupby(['PT_TYPE', 'PT_CODE', 'DAY_TYPE']).agg(
    august_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    august_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

sep_busstop_vol = transport_node_bus_202409.groupby(['PT_TYPE', 'PT_CODE', 'DAY_TYPE']).agg(
    september_tap_in = ('TOTAL_TAP_IN_VOLUME', 'sum'),
    september_tap_out = ('TOTAL_TAP_OUT_VOLUME', 'sum')
).reset_index()

#get passenger volume (tap in + tap out)
jul_busstop_vol['july_volume'] = jul_busstop_vol['july_tap_in'] + jul_busstop_vol['july_tap_out']
jul_busstop_vol = jul_busstop_vol.drop(columns = ['july_tap_in', 'july_tap_out'])

aug_busstop_vol['august_volume'] = aug_busstop_vol['august_tap_in'] + aug_busstop_vol['august_tap_out']
aug_busstop_vol = aug_busstop_vol.drop(columns = ['august_tap_in', 'august_tap_out'])

sep_busstop_vol['september_volume'] = sep_busstop_vol['september_tap_in'] + sep_busstop_vol['september_tap_out']
sep_busstop_vol = sep_busstop_vol.drop(columns = ['september_tap_in', 'september_tap_out'])

#merge to get all months volumes
busstop_volumes = pd.merge(jul_busstop_vol, aug_busstop_vol, on = ['PT_TYPE', 'PT_CODE', 'DAY_TYPE'], how = 'outer')
busstop_volumes = pd.merge(busstop_volumes, sep_busstop_vol, on = ['PT_TYPE', 'PT_CODE', 'DAY_TYPE'], how = 'outer')

#calc average passenger volume across the 3 months
busstop_volumes['average_passenger_volume'] = busstop_volumes[['july_volume', 'august_volume', 'september_volume']].mean(axis = 1)

#filter for weekday
weekday_busstop_volumes = busstop_volumes[busstop_volumes['DAY_TYPE'] == 'WEEKDAY']

print(weekday_busstop_volumes.head())

  PT_TYPE  PT_CODE DAY_TYPE  july_volume  august_volume  september_volume  \
0     BUS     1012  WEEKDAY      47934.0        47719.0           47150.0   
2     BUS     1013  WEEKDAY      31650.0        30577.0           29569.0   
4     BUS     1019  WEEKDAY      32331.0        31397.0           30008.0   
6     BUS     1029  WEEKDAY      44008.0        43114.0           43488.0   
8     BUS     1039  WEEKDAY      76900.0        71162.0           72208.0   

   average_passenger_volume  
0              47601.000000  
2              30598.666667  
4              31245.333333  
6              43536.666667  
8              73423.333333  


Combined Bus Data
- to be used for this remaining file

In [5]:
#merge trunkroute and weekday_busstop_volumes on BusStopCode = PT_CODE
combined_bus_data = pd.merge(trunkroute, weekday_busstop_volumes, left_on = 'BusStopCode', right_on = 'PT_CODE', how = 'inner')

#add column if the bus stop is MRT bus stop or not
combined_bus_data['MRTBusStop'] = combined_bus_data['Description'].apply(lambda x: 1 if 'Stn' in x or 'Int' in x else 0)

#add column to find MRTline of MRT bus stops
combined_bus_data['MRTLine'] = combined_bus_data.apply(
    lambda row: find_mrt_line(row['Description'], mrt_lines) if row['MRTBusStop'] == 1 else None, axis=1
)

#select only the relevant columns
combined_bus_data = combined_bus_data[['ServiceNo', 'Direction', 'StopSequence', 'BusStopCode', 'Description', 'MRTBusStop', 'MRTLine',
                                       'average_passenger_volume', 'Latitude', 'Longitude']]

print(combined_bus_data.head())


  ServiceNo  Direction  StopSequence  BusStopCode           Description  \
0        10          1             1        75009          Tampines Int   
1        10          1             2        76059  Opp Our Tampines Hub   
2        10          1             3        76069               Blk 147   
3        10          1             4        96289   Changi General Hosp   
4        10          1             5        96109          Opp Blk 3012   

   MRTBusStop                          MRTLine  average_passenger_volume  \
0           1  [Downtown Line, East-West Line]              1.899301e+06   
1           0                             None              1.127847e+05   
2           0                             None              1.112697e+05   
3           0                             None              1.187423e+05   
4           0                             None              4.042733e+04   

   Latitude   Longitude  
0  1.354076  103.943391  
1  1.352962  103.941652  
2  1.348753  1

parallelism_df 
- hardcoded to test, but to take data from parallelscore.ipynb
- filter for top 5 (can change)




In [6]:
parallelism_df = {
    'ServiceNo': ['63', '961M', '145', '36', '63A', '6', '127A', '973A', '68A', '88B'],
    'Direction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'ParallelismScore': [2.254552, 1.964276, 1.927232, 1.901650, 1.872716, 0.004168, 0.003957, 0.003083, 0.003008, 0.002643]
}

parallelism_df = pd.DataFrame(parallelism_df)
top_5_buses = parallelism_df.sort_values(by = 'ParallelismScore', ascending = False).head(5)

print(top_5_buses)



  ServiceNo  Direction  ParallelismScore
0        63          1          2.254552
1      961M          1          1.964276
2       145          1          1.927232
3        36          1          1.901650
4       63A          1          1.872716


Defining high and low passenger volume (TBC)
- up to us to decide

In [7]:
low_volume = combined_bus_data['average_passenger_volume'].quantile(0.50)


Classification function to remove/keep bus stops

considerations:
- used average passenger volume for all the bus stops in the inside and outside range to compare against our threshold of low_volume
    - may not be representative of all stops in between the MRT bus stops, for eg if indiv bus stops all have low passenger vol except 1 bus stop, do we remove that bus stop too?
- smth else to consider: maybe can use distance between each bus stop too (if v near each other can remove, if very far then cannot remove)
    

In [8]:
def process_bus_routes(combined_bus_data, top_buses, low_volume):

    #filter for top routes from the bus data and grp by bus
    top_routes = combined_bus_data[combined_bus_data['ServiceNo'].isin(top_buses['ServiceNo'])].copy()
    grouped_routes = top_routes.groupby(['ServiceNo', 'Direction'])

    #initialisation
    top_routes.loc[:, 'outcome'] = 'keep'
    final_routes = []

    for (service_no, direction), group in grouped_routes:

        group = group.sort_values(by = 'StopSequence')
        mrt_busstops = group[group['MRTBusStop'] == 1]
        
        #if route have < 2 MRT stops, keep the route
        if len(mrt_busstops) < 2:
            group.loc[:, 'outcome'] = 'keep'

        else:

            stop_sequences = mrt_busstops['StopSequence'].tolist()
            
            #iterate over each pairs of MRT stops
            for i in range(len(stop_sequences) - 1):

                #get seq number of both mrt bus stops
                first_mrt_busstop_sequence_no = stop_sequences[i]
                second_mrt_busstop_sequence_no = stop_sequences[i + 1]
                
                #calculate the average passenger volume for ALL stops in between the 1st and 2nd MRT bus stop
                in_between_stops = group[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no)]
                avg_in_between_volume = in_between_stops['average_passenger_volume'].mean()
                
                #if average vol is low, check outside range
                if avg_in_between_volume < low_volume:

                    #get seq number of before and after the 2 MRT bus stops
                    before_mrt = group[group['StopSequence'] < first_mrt_busstop_sequence_no]['average_passenger_volume'].mean()
                    after_mrt = group[group['StopSequence'] > second_mrt_busstop_sequence_no]['average_passenger_volume'].mean()
                    
                    #if outside range is low, remove all stops
                    if before_mrt < low_volume and after_mrt < low_volume:
                        group.loc[:, 'outcome'] = 'remove'

                    else:
                        #check if the MRT lines between the 1st and 2nd MRT busstops are matching
                        mrt_line_1 = mrt_busstops.iloc[i]['MRTLine'] if mrt_busstops.iloc[i]['MRTLine'] else []
                        mrt_line_2 = mrt_busstops.iloc[i + 1]['MRTLine'] if mrt_busstops.iloc[i + 1]['MRTLine'] else []
                        
                        mrt_lines_between = set(mrt_line_1).intersection(set(mrt_line_2))

                        #if MRT lines are matching, remove stops
                        if mrt_lines_between:
                            group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'remove'
                        
                        #else, keep it
                        else:
                            group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'keep'
                else:
                    #keep the in-between stops if volume is high
                    group.loc[(group['StopSequence'] > first_mrt_busstop_sequence_no) & (group['StopSequence'] < second_mrt_busstop_sequence_no), 'outcome'] = 'keep'

        final_routes.append(group)
    final_result = pd.concat(final_routes)

    return final_result



In [11]:

top_bus_routes = process_bus_routes(combined_bus_data, top_5_buses, low_volume)
print(top_bus_routes)

output_csv_path = "top_bus_routes.csv"
top_bus_routes.to_csv(output_csv_path, index = False)  # index=False to avoid saving row numbers
print(f"Top bus routes saved to {output_csv_path}")

      ServiceNo  Direction  StopSequence  BusStopCode             Description  \
2470        145          1             1        52009           Toa Payoh Int   
2471        145          1             2        52079        Opp Trellis Twrs   
2472        145          1             3        50161           Bef Jln Dusun   
2473        145          1             4        50171          Zhongshan Mall   
2474        145          1             5        50191     Bef Balestier Plaza   
...         ...        ...           ...          ...                     ...   
13259      961M          1            85         7371          Aft Kallang Rd   
13260      961M          1            86        60011    BENDEMEER STN EXIT A   
13261      961M          1            87        60059          Bef Kallang Pl   
13262      961M          1            88        80109  Opp Kallang Stn/Blk 2C   
13263      961M          1            89        80009       Lor 1 Geylang Ter   

       MRTBusStop          

extra: js to see the split between low passenger vol busstops vs total

In [10]:
def count_stops(final_result, low_volume):
    # Filter for stops with low passenger volume
    low_volume_stops = final_result[final_result['average_passenger_volume'] < low_volume]

    # Group by ServiceNo and Direction to count low-volume stops
    low_volume_count = low_volume_stops.groupby(['ServiceNo', 'Direction']).size().reset_index(name = 'low_volume_stop_count')

    # Group by ServiceNo and Direction to count total bus stops in each route
    total_stops_count = final_result.groupby(['ServiceNo', 'Direction']).size().reset_index(name = 'total_stop_count')

    # Merge the two counts
    result = pd.merge(total_stops_count, low_volume_count, on=['ServiceNo', 'Direction'], how = 'left')

    # Fill NaN values with 0 for routes that have no low-volume stops
    result['low_volume_stop_count'] = result['low_volume_stop_count'].fillna(0).astype(int)

    return result

# Use the function to count bus stops
bus_stop_counts = count_stops(top_bus_routes, low_volume)

# Print the results
print(bus_stop_counts)



  ServiceNo  Direction  total_stop_count  low_volume_stop_count
0       145          1                55                     20
1        36          1                61                     23
2        63          1               102                     35
3       63A          1                53                     17
4      961M          1                89                     55
